# 4. Escalas, eixos e legendas

Codificação visual &ndash; mapear dados para variáveis visuais como posição, tamanho, formato, ou cor &ndash; é o coração pulsante da visualização de dados. O motor que realmente performa o mapeamento é a escala (`scale`): a função que recebe os dados como entrada (o *domínio* da escala) e retorna um valor visual, como a posição de um pixel ou cor RGB, como saída (a *imagem* da escala). Evidentemente, a visualização é inútil se ninguém consegue entender o que ela quer transmitir! Complementando as marcas gráficas, um gráfico precisa de elementos de referência, ou *guias*, que permitem leitores lerem o gráfico. Guias como os *eixos* (que exibem escalas como intervalos no espaço) e *legendas* (que exibem escalas com uma faixa de cor, tamanho, ou formato), são heróis anônimos de visualização efetiva de dados!

Neste capítulo, nós vamos explorar as opções que o Altair oferece para permitir designs custumizados de mapeamento de escalas, eixos e legendas, usando e rodando um exemplo sobre a efetividade de antibióticos.

_Esse notebook é parte do [Currículo de Visualização de Dados](https://github.com/uwdata/visualization-curriculum)._

In [1]:
import pandas as pd
import altair as alt

## 4.1. Dados dos Antibióticos

Depois da Segunda Guerra Mundial, antibióticos foram considerados "drogas milagrosas", por serem um remédio fácil para doenças consideradas intratáveis. Para aprender qual droga funcionava mais efetivamente contra cada infecção bacteriana, a performance dos 3 antibióticos mais populares contra 16 das bactérias causadoras de infecções mais comuns foi coletada.

Nós vamos usar um conjunto de dados de antibióticos da [coleção do vega-datasets](https://github.com/vega/vega-datasets). No exemplo abaixo, nós vamos passar o URL diretamente para o Altair.

In [2]:
antibioticos = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/burtin.json'

Nós podemos primeiro carregar os dados com Pandas para ver o conjunto de dados inteiro e nos familiarizarmos com os campos de dados disponíveis:

In [3]:
pd.read_json(antibioticos)

,Bacteria,Penicillin,Streptomycin,Neomycin,Gram_Staining,Genus
0,Aerobacter aerogenes,870.000,1.00,1.600,negative,other
1,Bacillus anthracis,0.001,0.01,0.007,positive,other
2,Brucella abortus,1.000,2.00,0.020,negative,other
3,Diplococcus pneumoniae,0.005,11.00,10.000,positive,other
4,Escherichia coli,100.000,0.40,0.100,negative,other
5,Klebsiella pneumoniae,850.000,1.20,1.000,negative,other
6,Mycobacterium tuberculosis,800.000,5.00,2.000,negative,other
7,Proteus vulgaris,3.000,0.10,0.100,negative,other
8,Pseudomonas aeruginosa,850.000,2.00,0.400,negative,other
9,Salmonella (Eberthella) typhosa,1.000,0.40,0.008,negative,Salmonella


O valor numérico indicado na tabela é a [Concentração Mínima Inibitória (CMI)](https://pt.wikipedia.org/wiki/Concentra%C3%A7%C3%A3o_inibit%C3%B3ria_m%C3%ADnima), uma medida de eficácia do antibiótico, que representa a concentração de antibiótico (em microgramas por mililitro) necessária para previnir o crescimento da bactéria em laboratório. A reação da bactéria para um procedimento chamado [Teste de Gram](https://pt.wikipedia.org/wiki/T%C3%A9cnica_de_Gram) é descrita pela coluna `Gram_Staining`. Bactérias que ficam azul ou violeta no teste respondem positivamente (`positive`), enquanto as que não reagem assim são negativas (`negative`).

Enquanto examinamos diferentes visualizações desse conjunto de dados, se pergunte: O que podemos aprender sobre a eficácia relativa dos antibióticos? O que podemos aprender sobre as espécies bacterianas baseado na sua resposta ao antibiótico?

# 4.2 Ajustando escalas e eixos
## 4.2.1 Plotando a resistência a antibióticos: ajustando o tipo de escala

Vamos começar observando um gráfico de pontos simples da CMI para Neomicina (Neomycin).

In [4]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q')
)

alt.Chart(...)

*Podemos ver que os valores da CMI abrangem ordens de magnitude diferentes: a maior parte dos pontos está agrupada à esquerda, com alguns valores discrepantes maiores à direita.*

Por padrão, o Altair faz mapeamento linear entre os valores do domínio e a distância em pixels. Nesse caso, podemos ver que a distância entre 0 e 10 no eixo X em pixels é exatamente igual à distância entre 30 e 40. Enquanto isso é um padrão bom e o mais utilizado, no nosso gráfico, perdemos precisão ao observar os nossos dados, já que nosso eixo X é tão grande que o intervalo onde encontramos a maior parte dos nossos pontos fica achatado.

Para visualizar, então, nossos dados de maneira melhor, podemos aplicar uma transformação de escala no nosso eixo.


Para aplicar uma transformação de escala, ao criar o eixo, definimos seu atributo de escala (`scale`), usando o método `alt.Scale`, e definindo seu tipo (`type`).


Este seria o resultado se usássemos uma escala do tipo raiz quadrada (`type = 'sqrt'`). Distâncias em pixel agora correspondem à raiz quadrada da distância no domínio:

In [5]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q',
          scale = alt.Scale(type='sqrt'))
)

alt.Chart(...)

*Os pontos à esquerda estão melhor distribuídos agora, mas ainda temos discrepâncias bem grandes no gráfico.*

Vamos tentar usar uma escala [logarítmica](https://pt.wikipedia.org/wiki/Escala_logar%C3%ADtmica) (`type = 'log'`) agora:

In [6]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q',
          scale = alt.Scale(type='log'))
)

alt.Chart(...)

*Agora, nossos dados estão bem distribuídos ao longo do gráfico, e ainda conseguimos ver a diferença entre a CMI da Neomicina para diferentes bactérias.*

Numa escala linear, se uma distância de 5 unidades no seu eixo equivale a uma distância de 10 pixels, uma distância de 10 unidades vai equivaler a uma distância de 20 pixels, proporcionalmente. A escala logarítmica trabalha de maneira diferente, se baseando no mapeamento entre adição e multiplicação usando a propriedade matemática $log(a) + log(b) = log(ab)$. Como resultado disso, temos que se a distância visual entre dois pontos é de K pixels no seu gráfico, a distância numérica, nos seus dados, é na verdade uma proporção de K, onde K é a base escolhida para o nosso logaritmo (por padrão é 10, mas pode ser mudada definindo uma `base` na hora de usar o método `alt.Scale` com `type = 'log'`).

Visualize assim: temos 3 pontos A, B, C, que valem respectivamente 10, 100 e 1000 unidades. Em um eixo de um gráfico, na escala linear, se a distância em pixels entre A e B é de 10 pixels, a distância em pixels entre B e C é de 100 pixels, pois $C-B=10(B-A)$. Na escala logarítmica de base 10, no entanto, se a distância entre A e B é de 10 pixels, a distância entre B e C é de 10 pixels também, pois $\frac{C}{B} = \frac{B}{A}$.

## 4.2.2 Decorando um eixo

Para antibióticos, quanto menos usamos para matar uma bactéria, melhor esse antibiótico é. Considerando que gostamos normalmente de ver o melhor mais acima/à direita em um gráfico, para nos adequar a esse fator, teríamos que inverter o eixo X do nosso gráfico, de forma que as menores CMIs fiquem a esquerda e as maiores à direita.


Fazemos isso definindo seu atributo de ordenação (`sort`) como decrescente (`descending`):

In [7]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort = 'descending',
          scale = alt.Scale(type='log'))
)

alt.Chart(...)

*Nosso gráfico está começando a ficar confuso: estamos usando um eixo com escala logarítmica, em ordem inversa e sem indicação de unidades ainda!*

Vamos começar então a dar mais informações ao nosso gráfico adicionando um atributo de título (`title`) ao nosso eixo:

In [8]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort = 'descending',
          title = 'CMI da Neomicina (μg/ml, escala logarítmica reversa)',
          scale = alt.Scale(type='log'))
)

alt.Chart(...)

Bem melhor.

Podemos também mudar de lugar o nosso eixo. Por padrão, o Altair coloca o eixo X e suas informações na parte inferior do gráfico, mas podemos alterar isso adicionando um atributo de eixo (`axis`) no nosso código, usando o método `alt.Axis` e definindo sua orientação (`orient`) como superior (`'top'`):

In [9]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort = 'descending',
          axis = alt.Axis(orient = 'top'),
          title = 'CMI da Neomicina (μg/ml, escala logarítmica reversa)',
          scale = alt.Scale(type='log'))
)

alt.Chart(...)

O mesmo se aplica ao eixo Y, que normalmente está posicionado à esquerda mas pode ser posicionado à direita com a orientação `'right'`.

## 4.2.3 Comparação de antibióticos: Ajustar Grades, contagens de Tick e dimensionamento


_Como é que a neomicina se compara a outros antibióticos, como a estreptomicina e a penicilina?_

Para começar a responder a essas questões, podemos criar um gráfico de dispersão, adicionando um eixo vertical que codifica outro antibiótico e espelha nosso eixo x para a neomicina.

In [10]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Streptomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          title='CMI da Estreptomicina (μg/ml, escala logarítmica reversa)')
)

alt.Chart(...)

*Podemos ver que ambas a Neomicina e a Estreptomicina aparentam ser bem correlacionadas, já que as cepas bacteriais estudadas respondem de maneira similar a ambos antibióticos.*


Prosseguindo, vamos comparar a Neomicina e a Penicilina.

In [11]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)')
)

alt.Chart(...)

*Vemos agora uma resposta diferente: algumas bactérias respondem bem a um antibiótico enquanto outras não.*

Apesar do gráfico ser útil, podemos melhorar. Os eixos x e y usam a mesma unidade, mas tem dimensões diferentes (a largura do gráfico é maior que a altura) e domínios diferentes (0.001 até 100 no eixo x, e 0.001 até 1000 para o eixo y).

Vamos então igualar os eixos: podemos explicitar diretamente valores para altura (`height`) e largura (`width`) em pixels nas propriedades (`properties`) do gráfico e especificar o domínio (`domain`) na escala.


In [12]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)')
).properties(width=250, height=250)

alt.Chart(...)

_O gráfico resultante é mais equilibrado e menos propenso a erros de interpretação._

No entanto, as linhas de grade estão densas demais agora. Se quisermos remover completamente as linhas da grade (`grid`) do gráfico, podemos adicionar `grid=False` ao atributo do eixo. Mas e se, em vez disso, quisermos reduzir o número de marcas de seleção, por exemplo, incluindo apenas linhas para cada ordem de grandeza?

Para alterar o número de marcas, podemos especificar o número de marcações (`tickCount`) ao chamar o método `alt.Axis` no nosso código. A propriedade `tickCount` é tratada como uma sugestão para o Altair, a ser considerada juntamente com outros aspectos, tais como a utilização de intervalos agradáveis e de fácil utilização. Podemos não obter exatamente o número de marcas de escala que solicitamos, mas devemos obter algo próximo.

In [13]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis = alt.Axis(tickCount = 5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis = alt.Axis(tickCount = 5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)')
).properties(width=250, height=250)

alt.Chart(...)

Ao definir o `tickCount` para 5, obtemos o efeito desejado.

Os pontos do nosso gráfico de dispersão parecem um pouco pequenos. Vamos alterar o tamanho predefinido, definindo o tamanho (`size`) da marca circular ao criar nosso gráfico. Este valor de tamanho é a área da marca em pixels.

In [14]:
alt.Chart(antibioticos).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa )'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)')
).properties(width=250, height=250)

alt.Chart(...)

_Aqui, definimos a área da marca circular para 80 píxeis. Ajuste o valor como achar melhor._

# 4.3 Configurando as Legendas de Cor

## 4.3.1 Cor por Teste de Gram

Vimos acima que a Neomicina é mais eficaz para algumas bactérias enquanto a Penicilina é mais eficaz para outras. Mas, como podemos determinar qual antibiótico usar se não sabemos exatamente a espécie de bactéria com a qual estamos lidando? O teste de Gram serve como um método de separar classes de bactérias!

Vamos codificar o campo dos testes de Gram, `Gram_Staining`, no canal de cor (`color`) como um tipo de dado nominal:


In [15]:
alt.Chart(antibioticos).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Gram_Staining:N')
).properties(width=250, height=250)

alt.Chart(...)

Podemos ver que as bactérias que testam positivamente (`positive`) no teste de Gram parecem ser mais suscetíveis à Penicilina, enquanto a Neomicina é mais eficaz para bactérias que testam negativamente  (`negative`)!

O esquema de cores acima foi escolhido automaticamente para fornecer cores perceptivelmente distintas para comparações nominais (igual ou diferente). No entanto, podemos querer personalizar as cores utilizadas. Nesse caso, o teste de Gram resulta em colorações físicas distintas: as bactérias que testam negativo ficam rosa, e as que testam postivo ficam roxo.

Podemos determinar o uso dessas cores especificamente dentro da escala de `alt.Color`, especificando o domínio (`domain`) dos dados utilizados e o alcance (`range`) das cores selecionadas pelo Altair:


In [16]:
alt.Chart(antibioticos).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Gram_Staining:N',
          scale=alt.Scale(domain=['negative', 'positive'], range=['hotpink', 'purple'])
    )
).properties(width=250, height=250)

alt.Chart(...)

Por padrão, as legendas são posicionadas no lado direito do gráfico. Similar aos eixos, podemos alterar a orientação (`orient`) da legenda dentro do método `alt.Legend`:


In [17]:
alt.Chart(antibioticos).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Gram_Staining:N',
          scale=alt.Scale(domain=['negative', 'positive'], range=['hotpink', 'purple']),
          legend=alt.Legend(orient='left')
    )
).properties(width=250, height=250)

alt.Chart(...)

Também podemos remover completamente uma legenda especificando `legend=None`:



In [18]:
alt.Chart(antibioticos).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Gram_Staining:N',
          scale=alt.Scale(domain=['negative', 'positive'], range=['hotpink', 'purple']),
          legend=None
    )
).properties(width=250, height=250)

alt.Chart(...)

## 4.3.2 Cor por Espécie

Até agora, consideramos a eficácia dos antibióticos. Vamos mudar a abordagem e fazer uma pergunta diferente: o que a resposta aos antibióticos pode nos ensinar sobre as diferentes espécies de bactérias?

Para começar, vamos codificar `Bacteria` (um campo de dados nominal) usando o canal de `color`:


In [19]:
alt.Chart(antibioticos).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Bacteria:N')
).properties(width=250, height=250)

alt.Chart(...)

O resultado está um pouco confuso! Existem bactérias suficientes para que o Altair comece a repetir cores de sua paleta padrão de 10 cores para valores nominais.

Para usar cores personalizadas, podemos atualizar a propriedade `scale` da codificação de cor. Uma opção é fornecer valores explícitos de `domain` e `range` da escala para indicar mapeamentos de cores precisos por valor, como fizemos acima para `Gram Staining`. Outra opção é usar um esquema de cores alternativo. O Altair inclui uma variedade de esquemas de cores integrados. Para uma lista completa, consulte a documentação do esquema de cores do [Vega](https://vega.github.io/vega/docs/schemes/#reference).

Vamos tentar alternar para um esquema integrado de 20 cores, `tableau20`, e definir isso usando a propriedade de escala de esquema (`scheme`).


In [20]:
alt.Chart(antibioticos).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Nemicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Bacteria:N',
          scale=alt.Scale(scheme='tableau20'))
).properties(width=250, height=250)

alt.Chart(...)

Agora temos uma cor única para cada bactéria, mas o gráfico ainda está confuso. Entre outros problemas, a codificação não leva em conta bactérias que pertencem ao mesmo gênero. No gráfico acima, as duas cepas diferentes de Salmonella possuem matizes muito diferentes (verde-azulado e rosa), apesar de serem biologicamente próximas.

Para testar um `scheme` diferente, também podemos alterar o tipo de dado de nominal para ordinal. O `scheme` ordinal padrão usa tons de azul, variando do claro ao escuro:


In [21]:
alt.Chart(antibioticos).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Bacteria:O')
).properties(width=250, height=250)

alt.Chart(...)

Alguns desses tons de azul podem ser difíceis de distinguir.

Para obter cores mais diferenciadas, podemos experimentar alternativas ao esquema padrão `blues`. O esquema `viridis` varia tanto em matiz quanto em luminância:


In [22]:
alt.Chart(antibioticos).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Bacteria:O',
          scale=alt.Scale(scheme='viridis'))
).properties(width=250, height=250)

alt.Chart(...)

Bactérias do mesmo gênero agora possuem cores mais semelhantes do que antes, mas o gráfico ainda continua confuso. Há muitas cores, é difícil identificá-las com precisão na legenda, e duas bactérias podem ter cores semelhantes, mas pertencer a gêneros diferentes.


## 4.3.3. Colorindo por gênero

Vamos tentar colorir por gênero ao invés de colorir por bactéria. Para isso, adicionaremos o transformador `calculate` na criação do gráfico, separando o nome da bactéria por caracteres de espaço e selecionando o primeiro elemento da lista, assim tendo o gênero da bactéria. Podemos então codificar os gêneros resultantes, presentes no campo `Gênero`, utilizando o esquema de cores `tableau20`.

(Note que a base de dados "antibiotics" tem um campo de gênero, mas ignoraremos ele para poder fazer uma melhor exploração nas transformações de dados do Altair.)

In [23]:
alt.Chart(antibioticos).mark_circle(size=80).transform_calculate(
    Gênero='split(datum.Bacteria, " ")[0]'
).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Gênero:N',
          scale=alt.Scale(scheme='tableau20'))
).properties(width=250, height=250)

alt.Chart(...)

_Hmmm... Por mais que os dados estejam melhor agregados pelos gêneros, essa cacofonia de cores não parece muito útil._

_Se repararmos cuidadosamente em alguns do gráficos anteriores poderemos ver que apenas um grupo seleto de bactérias compartilham gênero com alguma outra: Salmonella, Staphylococcus, e Streptococcus. Para trazer um foco à nossa comparação adicionaremos cores apenas para esses gêneros repetidos._

Adicionemos outro transformador `calculate` que mantém o gênero se for um dos valores repetidos e caso contrário transforma em `"Outro"`.

Além disso, podemos adicionar codificadores de cores personalizados usando intervalos específicos com `domain` e `range` para definir as cores.

In [24]:
alt.Chart(antibioticos).mark_circle(size=80).transform_calculate(
  Split='split(datum.Bacteria, " ")[0]'
).transform_calculate(
  Gênero='indexof(["Salmonella", "Staphylococcus", "Streptococcus"], datum.Split) >= 0 ? datum.Split : "Outro"'
).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Gênero:N',
          scale=alt.Scale(
            domain=['Salmonella', 'Staphylococcus', 'Streptococcus', 'Outro'],
            range=['rgb(76,120,168)', 'rgb(84,162,75)', 'rgb(228,87,86)', 'rgb(121,112,110)']
          ))
).properties(width=250, height=250)

alt.Chart(...)

_Nós agora temos um gráfico muito mais revelador que só foi possivel graças às personalizações feitas nos eixos e legendas. Tire um momento para analisar o gráfico acima. Consegue perceber algum agrupamento surpreendente?_

_A parte superior esquerda apresenta um agrupamento de bactérias do gênero Streptococcus representados pela cor vermelha com uma bactéria do grupo "Outros" junta. Enquanto isso, na região do meio mais à direita observa-se outra Streptococcus posicionada longe de seus "primos". Devemos esperar que bactérias do mesmo gênero (e presumivelmente mais parecidas geneticamente) estejam mais próximas entre elas?_

Como ocorre eventualmente, a base de dados em questão contém erros. A base de dados se baseia nas designações de espécie utilizada no início dos anos 1950. No entanto, o consenso científico mudou desde então. O ponto cinza no canto superior esquerdo? Agora é considerado Streptococcus! Aquele ponto vermleho no meio à direita afastado dos demais? Não é mais considerado Streptococcus!

É claro que, por si só, essa base de dados não justifica essas reclassificações. Entretanto, os dados contém dicas valiosas para a biologia que foram ignoradas por décadas! Visualização de dados, quando utilizada por alguém habilidoso e curioso, pode ser uma arma poderosa para descobertas..

Esse exemplo reforça algo importante: **sempre tenha um pé atrás com seus dados!**

## 4.3.4. Colorindo por resposta a antibióticos

Nós também podemos utilizar o canal das cores `color` para codificar valores quantitativos. Mas é ipmortante ter em mente que o canal das cores não costuma ser tão eficiente para valores quantitativos quanto codificadores como posição e tamanho!

Aqui está um mapa de calor básico dos valores de CMI da Penicilina para cada bactéria. Usaremos uma marcação de retângulos (`rect`) e ordenar as bactérias por valores decrescente de CMI (de mais resistente a menos resistente).

In [25]:
alt.Chart(antibioticos).mark_rect().encode(
    alt.Y('Bacteria:N',
      sort=alt.EncodingSortField(field='Penicillin', op='max', order='descending')
    ),
    alt.Color('Penicillin:Q')
)

alt.Chart(...)

Nós podemos ainda melhorar esse gráfico combinando métodos vistos até aqui: uma escala logaritimica, uma mudança de orientação nos eixos, um esquema de cores personalizado (`plasma`), ajuste na contagem de marcações nos eixos e texto do titulo customizado. Nós também iremos usar opções de configurações para ajustar o posicionamento do título dos eixos e o alinhamento do título da legenda.

In [26]:
alt.Chart(antibioticos).mark_rect().encode(
    alt.Y('Bacteria:N',
      sort=alt.EncodingSortField(field='Penicillin', op='max', order='descending'),
      axis=alt.Axis(
        orient='right',     # Orienta o eixo à direita
        titleX=7,           # Define a posição X do título como 7 pixels à direita no gráfico
        titleY=-2,          # Define a posição Y do título como 2 pixels abaixo no gráfico
        titleAlign='left',  # Alinha o texto à esquerda
        titleAngle=0        # Desfaz a rotação padrão do título do eixo Y
      )
    ),
    alt.Color('Penicillin:Q',
      scale=alt.Scale(type='log', scheme='plasma', nice=True),
      legend=alt.Legend(titleOrient='right', tickCount=5),
      title='CMI da Penicilina (μg/ml)'
    )
)

alt.Chart(...)

Alternativamente, podemos remover o título do eixo e definir diretamente o `title` para todo o gráfico.

In [27]:
alt.Chart(antibioticos, title='Resistência a Penicilina de Cepas Bacterianas').mark_rect().encode(
    alt.Y('Bacteria:N',
      sort=alt.EncodingSortField(field='Penicillin', op='max', order='descending'),
      axis=alt.Axis(orient='right', title=None)
    ),
    alt.Color('Penicillin:Q',
      scale=alt.Scale(type='log', scheme='plasma', nice=True),
      legend=alt.Legend(titleOrient='right', tickCount=5),
      title='CMI da Penicilina (μg/ml)'
    )
).configure_title(
  anchor='start', # anchor and left-align title
  offset=5        # set title offset from chart
)

alt.Chart(...)

# 4.4. Sumário

Resumindo tudo que foi aprendido nos notebooks até aqui sobre codificação, transformação de dados e personalização, você agora deve estar preparado para fazer uma grande variedade de gráficos estátisticos. Agora você pode inserir o Altair em atividades do dia a dia para explorar dados e passar as mensagens escondidas neles.

Interessado em aprender mais sobre esse tópico?
- Comece com a [documentação do Altair sobre visualizações customizadas](https://altair-viz.github.io/user_guide/customization.html).
- Para uma discussão complementar sobre escalas de mapeamento veja ["Introducing d3-scale"](https://medium.com/@mbostock/introducing-d3-scale-61980c51545f).
- Para uma exploração mais profunda de como os eixos e as legendas podem ser personalizados pela biblioteca Vega (que é a base para Altair e Vega-Lite), veja ["A Guide to Guides: Axes & Legends in Vega"](https://beta.observablehq.com/@jheer/a-guide-to-guides-axes-legends-in-vega)
- Para uma história fascinante sobre a base de dados sobre antibióticos trabalhada esse notebok, veja ["That's Funny...", de Wainer e Lysen](https://www.americanscientist.org/article/thats-funny) na revista _American Scientist_.